# 关键词搜索
Kaggle竞赛题：https://www.kaggle.com/c/home-depot-product-search-relevance

In [20]:
%config ZMQInteractiveShell.ast_node_interactivity='all'
import sys
import os
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from nltk.stem.snowball import SnowballStemmer

In [25]:
file_path = '/Users/yangpan 1/works/recsys/nlp/search_relevance/input'
df_train = pd.read_csv(os.path.join(file_path, 'train.csv'), encoding="ISO-8859-1")
df_test = pd.read_csv(os.path.join(file_path, 'test.csv'), encoding="ISO-8859-1")
df_desc = pd.read_csv(os.path.join(file_path, 'product_descriptions.csv'))

In [26]:
df_train.shape
df_test.shape
df_desc.shape

(74067, 5)

(166693, 4)

(124428, 2)

In [31]:
df_train.head(2)

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.0
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.5


In [28]:
df_test.head(2)

,id,product_uid,product_title,search_term
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets


In [35]:
df_desc.head(2)

,product_uid,product_description
0,100001,"Not only do angles make joints stronger, they ..."
1,100002,BEHR Premium Textured DECKOVER is an innovativ...


In [33]:
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all.shape
df_all.head(3)

(240760, 5)

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.0
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.5
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.0


In [34]:
df_all = pd.merge(df_all, df_desc, how='left', on='product_uid')
df_all.shape
df_all.head(2)

(240760, 6)

,id,product_uid,product_title,search_term,relevance,product_description
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.0,"Not only do angles make joints stronger, they ..."
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.5,"Not only do angles make joints stronger, they ..."


### 文本预处理

In [36]:
stemmer = SnowballStemmer('english')

def str_stemmer(s):
    return " ".join([stemmer.stem(word) for word in s.lower().split()])

def str_common_word(str1, str2):
    return sum(int(str2.find(word)>=0) for word in str1.split())

In [ ]:
df_all['search_term'] = df_all['search_term'].map(lambda x:str_stemmer(x))
df_all['product_title'] = df_all['product_title'].map(lambda x:str_stemmer(x))
df_all['product_description'] = df_all['product_description'].map(lambda x:str_stemmer(x))